In [1]:
from get_cosine_sim import calculate_cosine, blending
import pandas as pd
import json

In [2]:
reviews=pd.read_csv('user_review_with_LLM.csv')
item=pd.read_csv('item_new.csv')
with open('ngcf.json', 'r') as json_file:
    ngcf = json.load(json_file)

In [3]:
sim_cache=calculate_cosine("review", 100, ngcf, reviews, item) #mode='review', 'LLM' #n: gcf로 추출하는 item 개수

Progress: 17465/17465


In [4]:
final_prediction=blending(100, 50, 0.5, ngcf, sim_cache) 

# 평가함수

### Recall

In [5]:
def calculate_recall (final_prediction, test):
    gt_user, gt_item = list(test['user_id']), list(test['item_id'])
    
    cnt=0

    for i in range(test.shape[0]):
        user=gt_user[i]
        if gt_item[i] in final_prediction[user]:
            cnt+=1
    
    correct = cnt
    gt = test.shape[0]
    recall = correct/gt
    
    return recall, correct, gt

In [6]:
test=pd.read_csv('graph_test.csv')
recall, recall_correct, recall_gt = calculate_recall(final_prediction, test)

### MAP

In [7]:
def calculate_map (k, final_prediction, test):
    n=test['user_id'].nunique()
    ap_list=[]
    gt = [(row.user_id, row.item_id) for row in test.itertuples(index=False)]
    for user in final_prediction.keys():
        cnt=0 #정답 세기
        user_sum=0
        for idx, item in enumerate(final_prediction[user]):
            if (user, item) in gt:
                cnt+=1
            user_sum+=(cnt/(idx+1))
        ap_list.append(user_sum/k)
    
    return sum(ap_list)/n

In [8]:
calculate_map(50, final_prediction, test)

2.2342839877725747e-05

In [11]:
recall_correct/recall_gt

0.0007072135785007072